In [1]:
%matplotlib inline
import numpy as np
from skimage import feature
import matplotlib.pyplot as plt
import keras, random, os
import keras.utils
from keras import losses
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Input, Embedding, Activation
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from sklearn.neighbors.kde import KernelDensity
from scipy import stats
from keras.models import load_model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
def get_labels():
    with open('labels.csv', 'r') as f:
        lines = f.readlines()
    
    label_dict = {}
    
    for line in lines:
        line = line.split(',')
        
        file = line[0].split('/')[-1]
        file = file.split('.')[0]
        
        label_dict[file] = []
        
        for coord in line[1:]:
            label_dict[file].append(float(coord))     
    
    return label_dict

In [3]:
label_dict = get_labels()

directory = 'data/'
files = os.listdir(directory)
files = [file for file in files if '.npy' in file]

scans = []
edges = []
labels = []
#scans = np.asarray(scans)

for file in files:
    name = file.split('.')[0]
    
    scan = np.load(directory+file)
    edge = np.array([feature.canny(slc, sigma=1) for slc in scan])
    label = label_dict[name]
    
    scans.append(scan)
    edges.append(edge)
    labels.append(label)

In [4]:
canidates = []
num_per_sample = 208
num_near_true = 25

#MAX = 0
#MIN = 0

val_split = 40

for e in range(len(edges)):
    
    history = set()
    shape = np.shape(edges[e])  
    
    for i in range(num_per_sample):
        valid = False

        while not valid:
            r1 = random.randint(0,shape[0]-33) #z
            r2 = random.randint(0,shape[1]-33) #x
            r3 = random.randint(0,shape[2]-33) #y
            
            if e <= val_split:
                if i == 0:
                    r1 = int(labels[e][2])
                    r2 = int(labels[e][0])
                    r3 = int(labels[e][1])

                elif i == num_near_true:
                    r1 = int(labels[e][5])
                    r2 = int(labels[e][3])
                    r3 = int(labels[e][4])

                #Get within say 6
                elif i > 0 and i < num_near_true:
                    r1 = random.randint(int(labels[e][2])-3, int(labels[e][2])+3) #z
                    r2 = random.randint(int(labels[e][0])-3, int(labels[e][0])+3) #x
                    r3 = random.randint(int(labels[e][1])-3, int(labels[e][1])+3) #y

                elif i > num_near_true and i < 2*num_near_true:

                    r1 = random.randint(int(labels[e][5])-3, int(labels[e][5])+3) #z
                    r2 = random.randint(int(labels[e][3])-3, int(labels[e][3])+3) #x
                    r3 = random.randint(int(labels[e][4])-3, int(labels[e][4])+3) #y

                while r1 < 0:
                    r1 += 1
                while r1 > shape[0]-33:
                    r1 -= 1
                while r2 < 0:
                    r2 += 1
                while r2 > shape[1]-33:
                    r2 -= 1
                while r3 < 0:
                    r3 += 1
                while r3 > shape[2]-33:
                    r3 -= 1
            
            coord = (r1,r2,r3)
            
        
            if (edges[e][r1][r2][r3] and coord not in history) or (e <= val_split and i < 2*num_near_true):
                history.add(coord)
                
                dif = np.array([labels[e][0] - r2, labels[e][1] - r3, labels[e][2] - r1, labels[e][3] - r2,
                               labels[e][4] - r3, labels[e][5] - r1])
                
                #m = np.max(dif)
                #m2 = np.min(dif)
                
                #if m > MAX:
                #    MAX = m
                    
                #if m2 < MIN:
                #    MIN = m2
                
                #So each canidate in the list contains the refrence scan #, the coord to sample, and the 6 dif points
                canidate = [e, coord, dif]
                
                canidates.append(canidate)
                
                valid = True

In [5]:
def get_sample_from(i):
    '''Takes in an index from the canidate list, and returns the label and the cropped scan in x'''
    
    s = canidates[i][0]
    coord = canidates[i][1]
    label = canidates[i][2]
    
    #Try normalizing the labels
    #label -= MIN
    #label /= (MAX-MIN)
    
    #Coord is in z,x,y like scans, but labels are x,y,z
    return np.expand_dims((scans[s][coord[0]:coord[0]+32, coord[1]:coord[1]+32, coord[2]:coord[2]+32]), axis=3), label

In [6]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, batch_size=16, dim=(32,32,32),
             n_classes=6, shuffle=True):
        
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def on_epoch_end(self):
 
        self.indexes = np.arange(len(self.list_IDs))
    
        if self.shuffle == True:
            
            #Just shuffle the order for each CT
            for i in range(len(self.indexes) // num_per_sample):
                np.random.shuffle(self.indexes[i*num_per_sample:(i+1)*num_per_sample])
            
            #np.random.shuffle(self.indexes)
            
            
    def __data_generation(self, list_IDs_temp):

        X = np.empty((self.batch_size, *self.dim, 1))
        y = np.empty((self.batch_size, 1, 1, 1, self.n_classes), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            
            X[i], y[i] = get_sample_from(ID)
    
        return X, y
    
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    
    def __getitem__(self, index):

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

In [7]:
params = {'dim': (32,32,32),
          'batch_size': 16,
          'n_classes': 6,
          'shuffle': True}

train = [i for i in range(val_split*num_per_sample)]
validation = [i for i in range(val_split*num_per_sample,len(canidates))]

training_generator = DataGenerator(train, **params)
validation_generator = DataGenerator(validation, **params)

In [8]:
def localization_net():

    model = Sequential()

    model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', input_shape=(32, 32, 32, 1)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same'))

    model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same'))

    model.add(Conv3D(filters=256, kernel_size=(3, 3, 3), padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(Conv3D(filters=256, kernel_size=(3, 3, 3), padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same'))

    model.add(Conv3D(filters=512, kernel_size=(4, 4, 4), padding='valid'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv3D(filters=512, kernel_size=(1, 1, 1), padding='same'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv3D(filters=6, kernel_size=(1, 1, 1), padding='same'))
    #model.add(BatchNormalization(axis=-1))

    #model.summary()
    return model

In [9]:
def iou_loss(y_true, y_pred):
    # iou loss for bounding box prediction
    # input is [x1, y1, z1, x2, y2, z2]

    
    x1_t = (K.transpose(y_true)[0])# * (MAX-MIN))
    y1_t = (K.transpose(y_true)[1])# * (MAX-MIN))
    z1_t = (K.transpose(y_true)[2])# * (MAX-MIN))
    x2_t = (K.transpose(y_true)[3])# * (MAX-MIN))
    y2_t = (K.transpose(y_true)[4])# * (MAX-MIN))
    z2_t = (K.transpose(y_true)[5])# * (MAX-MIN))
    
    x1_p = (K.transpose(y_pred)[0])# * (MAX-MIN))
    y1_p = (K.transpose(y_pred)[1])# * (MAX-MIN))
    z1_p = (K.transpose(y_pred)[2])# * (MAX-MIN))
    x2_p = (K.transpose(y_pred)[3])# * (MAX-MIN))
    y2_p = (K.transpose(y_pred)[4])# * (MAX-MIN))
    z2_p = (K.transpose(y_pred)[5])# * (MAX-MIN))

    # AOG = Area of Groundtruth box
    AoG = K.abs(x2_t - x1_t + 1) * K.abs(y2_t - y1_t + 1) * K.abs(z2_t - z1_t + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(x2_p - x1_p + 1) * K.abs(y2_p - y1_p + 1) * K.abs(z2_p - z1_p + 1)
    
    
    #print(K.eval(AoG))
    #print(K.eval(AoP))
    # overlaps are the co-ordinates of intersection box
    
    overlap_0 = K.maximum(x1_t, x1_p)
    overlap_1 = K.maximum(y1_t, y1_p)
    overlap_2 = K.maximum(z1_t, z1_p)
    overlap_3 = K.minimum(x2_t, x2_p)
    overlap_4 = K.minimum(y2_t, y2_p)
    overlap_5 = K.minimum(z2_t, z2_p)
    
    x = (overlap_3 - overlap_0 + 1)
    y = (overlap_4 - overlap_1 + 1)
    z = (overlap_5 - overlap_2 + 1)
    
    x = K.clip(x, 0.0, 1000000)
    y = K.clip(y, 0.0, 1000000)
    z = K.clip(z, 0.0, 1000000)
    
    intersection = x*y*z
    
    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())

    # loss for the iou value
    iou_loss = -K.log(iou)

    return iou_loss

def iou_metric(y_true, y_pred):
    # iou loss for bounding box prediction
    # input is [x1, y1, z1, x2, y2, z2]
  
    x1_t = (K.transpose(y_true)[0])# * (MAX-MIN)) + MIN
    y1_t = (K.transpose(y_true)[1])# * (MAX-MIN)) + MIN
    z1_t = (K.transpose(y_true)[2])# * (MAX-MIN)) + MIN
    x2_t = (K.transpose(y_true)[3])# * (MAX-MIN)) + MIN
    y2_t = (K.transpose(y_true)[4])# * (MAX-MIN)) + MIN
    z2_t = (K.transpose(y_true)[5])# * (MAX-MIN)) + MIN
    
    x1_p = (K.transpose(y_pred)[0])# * (MAX-MIN)) + MIN
    y1_p = (K.transpose(y_pred)[1])# * (MAX-MIN)) + MIN
    z1_p = (K.transpose(y_pred)[2])# * (MAX-MIN)) + MIN
    x2_p = (K.transpose(y_pred)[3])# * (MAX-MIN)) + MIN
    y2_p = (K.transpose(y_pred)[4])# * (MAX-MIN)) + MIN
    z2_p = (K.transpose(y_pred)[5])# * (MAX-MIN)) + MIN

    # AOG = Area of Groundtruth box
    AoG = K.abs(x2_t - x1_t + 1) * K.abs(y2_t - y1_t + 1) * K.abs(z2_t - z1_t + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(x2_p - x1_p + 1) * K.abs(y2_p - y1_p + 1) * K.abs(z2_p - z1_p + 1)
    
    
    #print(K.eval(AoG))
    #print(K.eval(AoP))
    # overlaps are the co-ordinates of intersection box
    
    overlap_0 = K.maximum(x1_t, x1_p)
    overlap_1 = K.maximum(y1_t, y1_p)
    overlap_2 = K.maximum(z1_t, z1_p)
    overlap_3 = K.minimum(x2_t, x2_p)
    overlap_4 = K.minimum(y2_t, y2_p)
    overlap_5 = K.minimum(z2_t, z2_p)
    
    x = (overlap_3 - overlap_0 + 1)
    y = (overlap_4 - overlap_1 + 1)
    z = (overlap_5 - overlap_2 + 1)
    
    x = K.clip(x, 0.0, 1000000)
    y = K.clip(y, 0.0, 1000000)
    z = K.clip(z, 0.0, 1000000)
    
    intersection = x*y*z
    
    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())
    
    return iou

In [10]:
model = localization_net()
#model.compile(loss=iou_loss, optimizer='adam', metrics = [iou_metric])
model.compile(loss='mean_squared_error', optimizer='adam', metrics = [iou_metric])
model.load_weights('weights.hdf5')

#model = load_model('model1.h5')

In [ ]:
checkpointer = ModelCheckpoint('/tmp/weights.hdf5', verbose=1, save_best_only=True)

In [ ]:
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True, workers=6, epochs=10, callbacks=[checkpointer])

Epoch 1/10
520/520 [==============================] - 842s 2s/step - loss: 246.8859 - iou_metric: 0.3260 - val_loss: 320.4315 - val_iou_metric: 0.3581

Epoch 00001: val_loss improved from inf to 320.43146, saving model to /tmp/weights.hdf5
Epoch 2/10
520/520 [==============================] - 863s 2s/step - loss: 238.1950 - iou_metric: 0.3206 - val_loss: 301.9840 - val_iou_metric: 0.3764

Epoch 00002: val_loss improved from 320.43146 to 301.98398, saving model to /tmp/weights.hdf5
Epoch 3/10
519/520 [============================>.] - ETA: 1s - loss: 222.5183 - iou_metric: 0.3262

In [ ]:
#model.save('model1.h5')
model.save_weights("weights.hdf5")

In [ ]:
model = localization_net()
model.compile(loss=iou_loss, optimizer='adam', metrics = [iou_metric])
#model.compile(loss='mean_squared_error', optimizer='adam', metrics = [iou_metric])
model.load_weights('weights.hdf5')

In [ ]:
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True, workers=6, epochs=20, callbacks=[checkpointer])

In [ ]:
def predict_point(predictions, to_pred):
    
    
    #print(predictions)
    
    #return np.round(np.mean(samples, axis=0))
    
    #o_dist = 10
    #sh = np.shape(edges[to_pred])

    #T = [pred for pred in predictions if pred[0] > -o_dist and pred[1] > -o_dist and pred[2] > -o_dist]
    #T = [pred for pred in T if pred[0] < sh[1]+o_dist and pred[1] < sh[2]+o_dist and pred[2] < sh[0]+o_dist]
    
    T=predictions
    
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(T)
    samples = kde.sample(n_samples=10000)
    
    #p = stats.mode(np.round(samples))[0][0]
    #print(p)
    p = np.mean(samples, axis=0)
    
    #print(p)
    return np.round(p)

def IoU(tl1,br1,tl2,br2):

    xA = max(tl1[0], tl2[0])
    yA = max(tl1[1], tl2[1])
    zA = max(tl1[2], tl2[2])
    
    xB = min(br1[0], br2[0])
    yB = min(br1[1], br2[1])
    zB = min(br1[2], br2[2])

    x = (xB - xA + 1)
    y = (yB - yA + 1)
    z = (zB - zA + 1)
    
    if (x < 0 or y < 0 or z < 0):
        return 0
    
    interArea = x*y*z
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (br1[0]-tl1[0]+1) * (br1[1]-tl1[1]+1) * (br1[2]-tl1[2]+1)
    boxBArea = (br2[0]-tl2[0]+1) * (br2[1]-tl2[1]+1) * (br2[2]-tl2[2]+1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

In [ ]:
def n(x):
    return (x - MIN) / (MAX-MIN)

def r(x):
    return (x*(MAX-MIN))-MIN


label1 = np.array([-2.0,1.0,1.0,6.0,5.0,7.0])
label2 = np.array([2.0,4.0,3.0,6.0,6.0,7.0])
label1 -= MIN
label1 /= (MAX-MIN)

label2 -= MIN
label2 /= (MAX-MIN)


#print(label1)
#print((-2-MIN) / (MAX-MIN))


#print(IoU([-2.0,1.0,1.0],[6.0,5.0,7.0],[2.0,4.0,3.0],[6.0,6.0,7.0]))
#x= iou_metric(label1, label2)

#K.eval(x) 


In [ ]:
def predict(to_pred):
    params = {'dim': (32,32,32),
              'batch_size': 10,
              'n_classes': 6,
              'shuffle': False}

    test = [x for x in range(to_pred*num_per_sample,(to_pred+1)*num_per_sample)]
    test_generator = DataGenerator(test, **params)
    predicted = model.predict_generator(test_generator)
    
    #Predicted points are scaled to / MAX so, bring them back to scale
    #predicted *= (MAX-MIN)
    #predicted += MIN
    
    predictionsTL = []
    predictionsBR = []

    #The actual labels...
    actualTL = [labels[to_pred][0], labels[to_pred][1], labels[to_pred][2]]
    actualBR = [labels[to_pred][3], labels[to_pred][4], labels[to_pred][5]]

    c = to_pred*num_per_sample
    for i in range(len(predicted)):

        #Get the coordinate of the guess
        coords = canidates[c+i][1]
        coord = np.array([coords[1],coords[2],coords[0]])
        
        tl,br = [],[]

        #Grab the predicted displacment vector for each point
        for y in range(6):
            if y < 3:
                tl.append(predicted[i][0][0][0][y])
            else:
                br.append(predicted[i][0][0][0][y])

        #Add the predictions as actual point + displacment
        predictionsTL.append(coord + np.array(tl))
        predictionsBR.append(coord + np.array(br))
        
        #print(IoU(coord+np.array(tl), coord+np.array(br), actualTL, actualBR))

        
    pTL = predict_point(predictionsTL, to_pred)
    dist1 = np.linalg.norm(pTL-actualTL)

    pTR = predict_point(predictionsBR, to_pred)
    dist2 = np.linalg.norm(pTR-actualBR)

    return dist1, dist2, IoU(pTL,pTR,actualTL,actualBR)

In [ ]:
dist1s = []
dist2s = []
ious = []

for i in range(40,59):
    d1,d2,u = predict(i)
    
    dist1s.append(d1)
    dist2s.append(d2)
    ious.append(u)

In [ ]:
print(np.mean(dist1s))
print(np.mean(dist2s))
print(np.mean(ious))